In [2]:
# Basic imports
import numpy as np
import pandas as pd

from pulp import *

### Récupérer les info

In [3]:
conso_df = pd.read_csv("./Consommations.csv", header=None)
apport_df = pd.read_csv("./Apports-hydro.csv", header=None)
sites_df = pd.read_csv("Sites.csv")

array_conso = conso_df.to_numpy()
array_apport = apport_df.to_numpy()
array_site = sites_df.to_numpy()

array_site = array_site[array_site[:, 0].argsort()]

### Donnée donné


In [4]:
cout_instal_onshore  = 168903 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)
cout_instal_offshore = 300336 # Coût *amorti sur un an* pour l'installation d'un MW éolien on-shore (euros/MW/an)

cout_instal_gaz = 94956 # Coût *amorti sur un an* pour l'installation d'un MW de centrale au gaz (euros/MW/an)
cout_fonct_gaz  = 65    # Coût de fonctionnement pour la production d'un MWh par centrale au gaz (euros/MWh)

stockage_pays = [0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6]

rend_turbinage = 0.85 

turbinage_pays = [8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101]
pompage_pays = [5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ]

In [5]:
# Donnée européenne
Smax = sum(stockage_pays) # Stockage de l'europe
Cmax = sum(sum(array_conso)) # Conso de l'europe
Amax = sum(sum(array_apport)) # Apport naturel de l'europe
Ppmax = sum(pompage_pays) # Puissance pompage de l'europe
Ptmax = sum(turbinage_pays) # Puissance turbinage de l'europe

# Question 4A & 4B


In [17]:
# création du problème
prob = LpProblem("Minimisation du coût d'installation des éoliennes", LpMinimize)

# variables
Wonshore = LpVariable("Puissance_installée_onshore", lowBound=0)
Woffshore = LpVariable("Puissance_installée_offshore", lowBound=0)
Ppump = LpVariable("Puissance_pompage", lowBound=0, upBound=rend_turbinage)
Pturbine = LpVariable("Puissance_turbinage", lowBound=0, upBound=rend_turbinage)

# fonction objectif
prob += cout_instal_onshore * Wonshore + cout_instal_offshore * Woffshore

# contraintes
prob += Wonshore + Woffshore + Pturbine - Ppump == Cmax
prob += 0 <= Ppump <= rend_turbinage
prob += 0 <= Pturbine <= rend_turbinage

prob.solve()

cout_total_prod = value(Wonshore) * cout_instal_onshore + value(Woffshore) * cout_instal_offshore
quantite_totale_conso = value(Wonshore) + value(Woffshore) + value(Pturbine)

prix_moyen = cout_total_prod / quantite_totale_conso

print("Prix moyen de l'électricité consommée:", prix_moyen)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/marco/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/934eaad3d3014e3eba317c290bc9c415-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/934eaad3d3014e3eba317c290bc9c415-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 24 ENDATA
Problem MODEL has 3 rows, 4 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-4) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 4.3748866e+14
After Postsolve, objective 4.3748866e+14, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 4.374886617e+14 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock 

# Question 4C